### Compute EKE from geostrophic velocity anomalies 

The Eddy Kinetic Energy computed here is

$$
\text{EKE} = \frac{1}{2} \left( u_a^2 + v_a^2\right) \ ,
$$

which has units of $\text{cm}^2 \text{s}^{-2}$ and where $u_a$ and $v_a$ are the zonal and meridional current velocity anomalies, respectively. 

**Altimetric product used here: two-sat vDT2018 from C3S**


In [1]:
%reload_ext autoreload
%autoreload 2

import xarray as xr
import os
import sys
sys.path.append('../')
import preprocessing_utils as pu

In [2]:
# Select which version of C3S data set: vDT2018 or vDT2021
# all C3S datasets are two-sat
version = 'vDT2018' 

# Specify the path to the raw data folder
raw_data_folder = f'../../raw_c3s_data/{version}/'

# Specify the output folder for processed data
output_folder = f'../../processed_data/EKE_GLOBAL/EKE_TWO_SAT_C3S_{version}/'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    


In [3]:
# Loop through the year folders
listd = os.listdir(raw_data_folder)
listd.sort()

for year_folder in listd:
    year_path = os.path.join(raw_data_folder, year_folder)

    # Check if it's a directory
    if os.path.isdir(year_path):
        
        # Get a list of netCDF files within the year folder
        files = [os.path.join(year_path, file) for file in os.listdir(year_path) if file.endswith('.nc')]

        # Sort the list of files
        files.sort()   
        
        # Loop through the netCDF files
        for file in files:
            # Open the netCDF dataset
            ds = xr.open_dataset(file)
    
            lon_name = 'longitude'  

            # Save lon attrs
            lon_attrs = ds[lon_name].attrs
            
            # Adjust lon values to make sure they are within (-180, 180)
            ds['_longitude_adjusted'] = xr.where(
                                                ds[lon_name] > 180,
                                                ds[lon_name] - 360,
                                                ds[lon_name])
            
            # Set attributes for the adjusted DataArray
            ds['_longitude_adjusted'].attrs = lon_attrs
    

            # reassign the new coords to as the main lon coords
            # and sort DataArray using new coordinate values
            ds = (
                ds
                .swap_dims({lon_name: '_longitude_adjusted'})
                .sel(**{'_longitude_adjusted': sorted(ds._longitude_adjusted)})
                .drop(lon_name))

            ds = ds.rename({'_longitude_adjusted': lon_name})
            
            # Remove valid_max and valid_min attributes from longitude and latitude DataArray
            # so that the EKE file doesn't inherit them
            attributes_to_remove = ['valid_max', 'valid_min']

            for attr in attributes_to_remove:
                ds['longitude'].attrs.pop(attr, None)
                ds['latitude'].attrs.pop(attr, None)
             
            # Extract date information from the file name 
            dirpath, filename = os.path.split(file)
            base, ext         = os.path.splitext(filename)
            parts             = base.split("_")
            newfilename       = "_".join(['TWO_SAT_C3S', parts[-1], parts[-2]])
            
            # Specify the output file path
            outfile = f'EKE_{newfilename}.nc'
            outpath = os.path.join(output_folder, outfile)
            #print(outpath)
            
            # Compute the EKE and save to the output file
            pu.compute_eke(ds, outfile, output_folder)

            # Close the netCDF dataset
            ds.close()
            
            